In [1]:
# Upgrade pip
%pip install --upgrade pip

# Install tensorflow and keras 
%pip install --no-cache-dir tensorflow==2.20.0 tf-keras==2.20.0

# Install torch and torchdata
%pip install --no-deps torch torchdata --quiet

%pip install -U \
    datasets \
    transformers \
    accelerate \
    evaluate \
    bert-score \
    peft --quiet

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.3/332.0 MB ? eta -:--:--
   ---------------------------------------- 0.8/332.0 MB 2.0 MB/s eta 0:02:46
   ---------------------------------------- 1.0/332.0 MB 2.0 MB/s eta 0:02:43
   ---------------------------------------- 1.3/332.0 MB 1.7 MB/s eta 0:03:10
   ---------------------------------------- 1.6/332.0 MB 1.5 MB/s eta 0:03:44
   ---------------------------------------- 1.6/332.0 MB 1.5 MB/s eta 0:03:44
   ---------------------------------------- 1.8/332.0 MB 1.3 MB/s eta 0:04:12
   ---------------------------------------- 2.1/332.0 MB 1.3 MB/s eta 0:04:23
   ---------------------------------------- 2.4/332.0 MB 1.3 MB/s eta 0:04:22
   ---------------------------------------- 2.6/332.0 MB 1.2 MB/s eta 0:04:32
   ----------------

Reason for being yanked: <none given>


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np
import json

## Set up the dataset

In [3]:
train_dataset = []
test_dataset = []

with open('Datasets/PlantVQA_Agriculture-Plan-Diseases-QA-Pairs-Dataset(Preprocessed)/final_dataset.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # skip empty lines
            train_dataset.append(json.loads(line))

print(f"Loaded {len(train_dataset)} records.")
print(train_dataset[0]) 

Loaded 13926 records.
{'question': 'What is the primary subject of this image?', 'answer': 'This is a background image without a plant leaf.', 'split': 'train'}


In [4]:
with open('Datasets/PlantVQA_Agriculture-Plan-Diseases-QA-Pairs-Dataset(Preprocessed)/test.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        if line.strip():  # skip empty lines
            test_dataset.append(json.loads(line))

print(f"Loaded {len(test_dataset)} records.")
print(test_dataset[0])

Loaded 474 records.
{'question': "Can you trace the cause of this vegetation's appearance?", 'answer': 'The mosaic pattern of light and dark green areas is a classic symptom of ToMV.', 'split': 'test'}


## Load the model

In [ ]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

## Define Tokernizer

In [ ]:
def tokenize_function(example):
    start_prompt = 'Answer Following Question\n\n'
    end_prompt = '\n\nAnswer: '
    prompt = [start_prompt + question + end_prompt for question in example["question"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["answer"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# tokenize train dataset
tokenized_train_datasets = tokenize_function({
    "question": [item["question"] for item in train_dataset],
    "answer": [item["answer"] for item in train_dataset]
})

# tokenize test dataset
tokenized_test_datasets = tokenize_function({
    "question": [item["question"] for item in test_dataset],
    "answer": [item["answer"] for item in test_dataset]
})